In [1]:
import pandas as pd
import itertools
import numpy as np

from pyECLAT import ECLAT

In [2]:
# https://www.kaggle.com/code/annadurbanova/market-basket-optimization-eclat-prac/data 
data = pd.read_csv("Market_Basket_Optimisation.csv.xls", header=None)

In [3]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from operator import itemgetter
products = data.stack()
transaction_indexes = [i[0] for i in products.index.tolist()]
df = pd.DataFrame(products)#,transaction_indexes)
unique_products = pd.DataFrame(df[0].unique())
from operator import itemgetter

unique_products['transactions'] = unique_products.apply(lambda x: list(map(itemgetter(0),df[df[0] == x[0]].index.tolist())), axis=1)

In [5]:
unique_products

,0,transactions
0,shrimp,"[0, 16, 20, 23, 69, 91, 107, 108, 110, 125, 14..."
1,almonds,"[0, 94, 101, 102, 105, 118, 253, 276, 414, 467..."
2,avocado,"[0, 3, 23, 25, 31, 45, 48, 67, 107, 141, 158, ..."
3,vegetables mix,"[0, 151, 210, 234, 245, 338, 408, 529, 641, 65..."
4,green grapes,"[0, 107, 237, 250, 307, 357, 472, 528, 554, 63..."
...,...,...
115,burger sauce,"[522, 604, 718, 808, 895, 906, 1082, 1103, 119..."
116,oatmeal,"[533, 1098, 1147, 1200, 1251, 1527, 1751, 1848..."
117,asparagus,[694]
118,cream,"[826, 1306, 2376, 3919, 3952, 4256, 5044]"


In [6]:
dataFrame = [[[unique_products.iloc[i][0]],unique_products.iloc[i]["transactions"]] for i in range(len(unique_products))]

In [7]:
num_transactions = 7501 
min_support = 0.05
min_combination = 1

# slightly more efficient way to calculate support threshold
support_threshold = min_support * num_transactions

In [8]:
def union(l1,l2):
    union = l1 + l2
    return list(dict.fromkeys(union))

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [9]:
max_k = 10
F = [[] for i in range(max_k)]

def bottom_up(S):

    T = []

    for i in range(len(S)):
        
        Ti = []
        
        for j in range(i+1, len(S)):
            R = union(S[i][0], S[j][0])
            trans_R = intersection(S[i][1], S[j][1])

            if len(trans_R) >= support_threshold:
                Ti.append([R,trans_R])
                F[len(R)].append(R)

        T.append(Ti)
    
    for i in T:
        if i != []:
            bottom_up(i)

In [20]:
# libraries and timer
from apyori import apriori
from pyECLAT import ECLAT
import pyfpgrowth

# conversion from kaggle, apriori preprocessing
transactions = []
for i in range(0, 5000):
    transactions.append([str(data.values[i,j]) for j in range(0, 20)])
    
# pyeclat preprocessing
data = pd.read_csv("Market_Basket_Optimisation.csv.xls", header=None)
eclat = ECLAT(data=data)

max_combination = 3

In [14]:
%%time
rules = apriori(transactions = transactions, min_support = min_support)

Wall time: 0 ns


In [22]:
%%time
bottom_up(dataFrame)

Wall time: 3.42 s


In [21]:
%%time 
rule_indices, rules_support = eclat.fit(min_support=min_support,
                                                 min_combination=min_combination,
                                                 max_combination=max_combination,
                                                 separator=' & ',
                                                 verbose=False)

Wall time: 16.2 s


In [18]:
%%time
frequent_itemsets = pyfpgrowth.find_frequent_patterns(transactions=transactions,support_threshold=min_support)

Wall time: 49.5 s
